# Imports

In [2]:
import os
import sys
import inspect
import glob

import torch
from PIL import Image
import torchvision.transforms as transforms

sys.path.insert(0,'..')
import process_section
import clipscore

from torchmetrics.multimodal import CLIPScore

# Existing clipscore implementation

In [23]:
# Change num_workers to 0 to account for local issue

def process_images(image_dir, model, device):
    image_paths = glob.glob(f'{image_dir}/*/*.jpg')

    # a dictionary
    image_feats_lookup = clipscore.extract_all_images(
        image_paths, model, device, batch_size=64, num_workers=0)

    return image_feats_lookup

In [24]:
model, device = clipscore.get_clip_mdl()
image_feats_lookup = process_images('imgdir', model, device)

100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.61s/it]


In [19]:
def gen_clip_score(img_name, prompt, image_feats_lookup, model, device):
    image_set = [img_name for _ in range(6)]
    segments = [('s1', prompt)]
    result = process_section.calc_clip(segments, image_set, image_feats_lookup, model, device)
    return result[0][0]

In [20]:
result = gen_clip_score('./imgdir/inner/cat.jpg', 'This is a cat', image_feats_lookup, model, device)
print(result)

100%|████████████████████████████████████████████████████████████████████████████| 1/1 [00:08<00:00,  8.12s/it]

0.6560731


# TorchMetrics implementation

In [43]:
metric = CLIPScore(model_name_or_path="openai/clip-vit-base-patch32")

In [45]:
def img2tensor(imgname):
    image = Image.open(imgname)
    transform = transforms.Compose([transforms.PILToTensor()])
    img_tensor = transform(image)
    return img_tensor

In [77]:
score = metric(img2tensor('./imgdir/inner/cat.jpg'), "This is a cat")

In [78]:
score.detach()

tensor(26.6762)